<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Mw-Pradeep/Bioco_Mw_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lasting av nødvendige bibliotek og pakker

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.8 MB/s eta 0:00:00


Importering av nødvendig bibliotek og pakker

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, r2_score

import optuna
from optuna.visualization import plot_optimization_history

Importering av relevant data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Velger første kolonne med dato og tid som index
mw_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data.csv',
                      header=0,
                      sep=',',
                      index_col=0)

# Formatterer index til riktig format og datatype
mw_data.index = pd.to_datetime(mw_data.index,
                               format='%Y-%m-%d %H:%M:%S')

Mounted at /content/drive


Tilfeldighetsfrø

In [ ]:
random_seed = 123

Beskrivelse av datasett

In [ ]:
mw_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29136 entries, 2022-10-31 17:37:00 to 2023-06-14 01:06:00
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29136 non-null  int64  
 1   EnzymeType_A2  29136 non-null  int64  
 2   EnzymeType_B   29136 non-null  int64  
 3   EnzymeType_C   29136 non-null  int64  
 4   EnzymeType_D   29136 non-null  int64  
 5   EnzymeType_E   29136 non-null  int64  
 6   RawMatFlow     29136 non-null  float64
 7   NIRfat         29136 non-null  float64
 8   NIRash         29136 non-null  float64
 9   NIRwater       29136 non-null  float64
 10  TT07           29136 non-null  float64
 11  TT08           29136 non-null  float64
 12  PT03           29136 non-null  float64
 13  TT20           29136 non-null  float64
 14  TT09           29136 non-null  float64
 15  TT12           29136 non-null  float64
 16  Mw             460 non-null    float64
dtypes: float64(11),

# Oppdeling av datasett for trening og testing

### Alternativ 1: Fordelt på Enzymtyper

In [ ]:
 def oppdeling_alternativ_1(df, test_size=0.2, random_seed=123):

  # Responsvariabel
  responsvariabel = df.columns[-1]

  # Deler opp markert og umarkert del av datasettet
  data_markert = df.dropna(subset=[responsvariabel])
  data_umarkert = df[df[responsvariabel].isna()]

  # Legger til kategorisk variabel for enzymtype til fordeling av datsettet
  data_markert['EnzymType'] = data_markert.filter(like='EnzymeType_').idxmax(axis=1).str.split('_').str[1].astype('category')

  # Splitter datasett i dataasett og kategorisk enzymtype
  enzymetypes = data_markert['EnzymType']
  data_markert.drop(columns=['EnzymType'], inplace=True)

  # Splitter trening og testdata etter enzymtype
  trening_markert, test_markert, _, _ = train_test_split(data_markert,
                                                          enzymetypes,
                                                          test_size=test_size,
                                                          stratify=enzymetypes,
                                                          random_state=random_seed)

  # Definerer trening og testsett for data
  testdata = test_markert

  felles_indeks = df.index.intersection(testdata.index)
  treningsdata = df.drop(felles_indeks)

  return treningsdata, testdata

# NIR målinger inkludert

## Oppdeling av trening og testsett

### Alternativ 1: Fordelt på Enzymtype

In [ ]:
mw_trening, mw_test = oppdeling_alternativ_1(mw_data)
mw_trening_markert = mw_trening.dropna(subset=["Mw"])

<ipython-input-6-e7454339f6c4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_markert['EnzymType'] = data_markert.filter(like='EnzymeType_').idxmax(axis=1).str.split('_').str[1].astype('category')
<ipython-input-6-e7454339f6c4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_markert.drop(columns=['EnzymType'], inplace=True)


In [ ]:
X_trening = mw_trening_markert.iloc[:, :-1]
X_test = mw_test.iloc[:, :-1]
y_trening = mw_trening_markert.iloc[:, -1]
y_test = mw_test.iloc[:, -1]

## RandomForestRegressor

Hyperparamter optimalisering med Optuna

In [ ]:
def objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rf_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rf_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rf_pipeline.predict(X_test)

  mse_test = mean_squared_error(y_test, y_test_prediksjon)

  return mse_test

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=100)

plot_optimization_history(study)

[I 2024-02-08 15:46:42,170] A new study created in memory with name: no-name-edad19dc-53c4-4b2e-b431-41ec71c8a125
[I 2024-02-08 15:46:42,454] Trial 0 finished with value: 344322.7978575765 and parameters: {'n_estimators': 103, 'max_depth': 6}. Best is trial 0 with value: 344322.7978575765.
[I 2024-02-08 15:46:42,956] Trial 1 finished with value: 343126.0461370651 and parameters: {'n_estimators': 169, 'max_depth': 6}. Best is trial 1 with value: 343126.0461370651.
[I 2024-02-08 15:46:43,365] Trial 2 finished with value: 321392.49997396057 and parameters: {'n_estimators': 133, 'max_depth': 7}. Best is trial 2 with value: 321392.49997396057.
[I 2024-02-08 15:46:43,697] Trial 3 finished with value: 3043913.5425365805 and parameters: {'n_estimators': 180, 'max_depth': 1}. Best is trial 2 with value: 321392.49997396057.
[I 2024-02-08 15:46:44,341] Trial 4 finished with value: 309619.08692332206 and parameters: {'n_estimators': 168, 'max_depth': 10}. Best is trial 4 with value: 309619.0869233

Evauleringer av beste modell

In [ ]:
def detailed_objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rf_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rf_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rf_pipeline.predict(X_test)
  y_trening_prediksjon = rf_pipeline.predict(X_trening)

  mse_test = mean_squared_error(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)

  mse_trening = mean_squared_error(y_trening, y_trening_prediksjon)
  r2_trening = r2_score(y_trening, y_trening_prediksjon)

  return mse_test, r2_test, mse_trening, r2_trening

mse_test_resultat = detailed_objective(study.best_trial)[0]
r2_test_resultat = detailed_objective(study.best_trial)[1]
mse_trening_resultat = detailed_objective(study.best_trial)[2]
r2_trening_resultat = detailed_objective(study.best_trial)[3]

print("Beste hyperparametere for RandomForestRegressor: {}".format(study.best_params))
print("Test: MSE: {}, og R^2: {}".format(mse_test_resultat, r2_test_resultat))
print("Trening: MSE: {}, og R^2: {}".format(mse_trening_resultat, r2_trening_resultat))

Beste hyperparametere for RandomForestRegressor: {'n_estimators': 119, 'max_depth': 9}
Test: MSE: 304150.56270906975, og R^2: 0.9460160945821272
Trening: MSE: 61240.625341091145, og R^2: 0.9893721909646818


# NIR målinger ekskludert

In [ ]:
mw_data_uten_NIR = mw_data.drop(columns=['NIRfat', 'NIRwater', 'NIRash'])
mw_data_uten_NIR.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29136 entries, 2022-10-31 17:37:00 to 2023-06-14 01:06:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29136 non-null  int64  
 1   EnzymeType_A2  29136 non-null  int64  
 2   EnzymeType_B   29136 non-null  int64  
 3   EnzymeType_C   29136 non-null  int64  
 4   EnzymeType_D   29136 non-null  int64  
 5   EnzymeType_E   29136 non-null  int64  
 6   RawMatFlow     29136 non-null  float64
 7   TT07           29136 non-null  float64
 8   TT08           29136 non-null  float64
 9   PT03           29136 non-null  float64
 10  TT20           29136 non-null  float64
 11  TT09           29136 non-null  float64
 12  TT12           29136 non-null  float64
 13  Mw             460 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 4.3 MB


## Oppdeling av trening og testsett

### Alternativ 1: Fordelt på Enzymtyper

In [ ]:
mw_trening, mw_test = oppdeling_alternativ_1(mw_data_uten_NIR)
mw_trening_markert = mw_trening.dropna(subset=["Mw"])

X_trening = mw_trening_markert.iloc[:, :-1]
X_test = mw_test.iloc[:, :-1]
y_trening = mw_trening_markert.iloc[:, -1]
y_test = mw_test.iloc[:, -1]

<ipython-input-6-e7454339f6c4>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-e7454339f6c4>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## RandomForestRegressor

In [ ]:
def objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rf_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rf_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rf_pipeline.predict(X_test)

  mse_test = mean_squared_error(y_test, y_test_prediksjon)

  return mse_test

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=100)

plot_optimization_history(study)

[I 2024-02-08 15:56:36,971] A new study created in memory with name: no-name-2d3c7bfa-7e29-4e6d-9cbf-32167bc49d62
[I 2024-02-08 15:56:37,312] Trial 0 finished with value: 658685.5200800243 and parameters: {'n_estimators': 119, 'max_depth': 9}. Best is trial 0 with value: 658685.5200800243.
[I 2024-02-08 15:56:37,662] Trial 1 finished with value: 1752283.341717364 and parameters: {'n_estimators': 179, 'max_depth': 3}. Best is trial 0 with value: 658685.5200800243.
[I 2024-02-08 15:56:37,952] Trial 2 finished with value: 1096690.3855943794 and parameters: {'n_estimators': 127, 'max_depth': 4}. Best is trial 0 with value: 658685.5200800243.
[I 2024-02-08 15:56:38,368] Trial 3 finished with value: 655842.2450160381 and parameters: {'n_estimators': 145, 'max_depth': 10}. Best is trial 3 with value: 655842.2450160381.
[I 2024-02-08 15:56:38,790] Trial 4 finished with value: 660049.0691353395 and parameters: {'n_estimators': 152, 'max_depth': 10}. Best is trial 3 with value: 655842.2450160381

In [ ]:
def detailed_objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rf_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rf_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rf_pipeline.predict(X_test)
  y_trening_prediksjon = rf_pipeline.predict(X_trening)

  mse_test = mean_squared_error(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)

  mse_trening = mean_squared_error(y_trening, y_trening_prediksjon)
  r2_trening = r2_score(y_trening, y_trening_prediksjon)

  return mse_test, r2_test, mse_trening, r2_trening

mse_test_resultat = detailed_objective(study.best_trial)[0]
r2_test_resultat = detailed_objective(study.best_trial)[1]
mse_trening_resultat = detailed_objective(study.best_trial)[2]
r2_trening_resultat = detailed_objective(study.best_trial)[3]

print("Beste hyperparametere for RandomForestRegressor: {}".format(study.best_params))
print("Test: MSE: {}, og R^2: {}".format(mse_test_resultat, r2_test_resultat))
print("Trening: MSE: {}, og R^2: {}".format(mse_trening_resultat, r2_trening_resultat))

Beste hyperparametere for RandomForestRegressor: {'n_estimators': 137, 'max_depth': 10}
Test: MSE: 651503.567942855, og R^2: 0.8843641561009515
Trening: MSE: 89700.96796595304, og R^2: 0.984433131560698
